<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<a href="http://mng.bz/orYv">《从零构建大语言模型》</a>一书的补充代码，作者：<a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>代码仓库：<a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 第二章：文本处理
本notebook需要使用的库和对应版本

In [46]:
from importlib.metadata import version

print("torch version: ", version('torch'))
print("tiktoken version: ", version('tiktoken'))


torch version:  2.2.1+cu118
tiktoken version:  0.6.0


- 本章介绍数据准备和采样，为大语言模型(LLM)的输入数据做好"准备"

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="800px">
</p>

## 2.1 理解词嵌入
- 本节没有代码
- 存在各种形式的嵌入，如视频嵌入、音频嵌入等，LLM中目前主要聚焦在文本嵌入。对于嵌入/embeddings可以理解为一种可表示原始数据对象的向量数据，因为训练过程就是一个计算的过程，需要将各类训练数据转换为可计算的形式。

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="600px">
</p>

- LLMs在高维空间中与嵌入交互，如上千个维度
- 人类只能观测到1、2、3维空间，所以不能将高维空间可视化，但下图中二维空间可以大致表明嵌入空间的概念

<p align="center">
  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="500px">
</p>

## 2.2 文本分词
- 本节对文本进行分词，或者理解为标记化处理，即将一段文本序列分解为更小的单元，如单个词或标点符号。下图中是将一个句子按单词进行分词，分词后每个单词对应一个id，可以理解为该单词在整个vocab中的索引，也可以理解为分类任务中的一个类别标签，因为是数值，可以通过embedding层转换为嵌入向量

<p align="center">
  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="500px">
</p>

- 加载准备处理的数据
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict)是一个公开、无版权的短篇小说

In [ ]:
import os
import requests

if not os.path.exists("the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)


# The book originally used the following code below
# However, urllib uses older protocol settings that
# can cause problems for some readers using a VPN.
# The `requests` version above is more robust
# in that regard.

"""
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)
"""

<br>

---

<br>

#### 故障排除 SSL 证书错误

- 一些读者在VSCode或Jupyter中运行`urllib.request.urlretrieve`时遇到了ssl.SSLCertVerificationError: `SSL: CERTIFICATE_VERIFY_FAILED`错误。
- 这通常意味着Python的证书包已过期。

**解决方案**

- 使用Python ≥ 3.9；可以通过执行以下代码来检查您的Python版本：
```python
import sys
print(sys.__version__)
```
-升级证书包：
  - pip: `pip install --upgrade certifi`
  - uv: `uv pip install --upgrade certifi`
- 升级后重启Jupyter内核。
- 如果在执行上一个代码单元格时仍然遇到`ssl.SSLCertVerificationError`错误，请查看[GitHub 上的更多信息](https://github.com/rasbt/LLMs-from-scratch/pull/403)

<br>

---

<br>

In [48]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters: ", len(raw_text))
print(raw_text[:99])

Total number of characters:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是对文本进行分词和嵌入化，以其可作为LLMs的输入
- 基于一些简单的示例文本开发一个简单的分词器，之后可以应用到上面的文本中
- 以下正则表达式将按空白字符进行分割

In [49]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 不仅想按空白字符分割，还想按逗号和句号分割，所以让修改正则表达式以实现这一点

In [50]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 上述实现会产生空字符，将其删除

In [51]:
# 消除每个元素前后的空字符，然后将所有为空字符的元素过滤掉
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 上述实现不错，但是还要处理如句号、问号等标点符号

In [52]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 上述实现的分词方式已经可以应用到原始文本了

<p align="center">
  <img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">
</p>

In [53]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 查看所有tokens的数量

In [54]:
print(len(preprocessed))

4690


## 2.3 将tokens隐射为token IDs
- tokens转换为token IDs便于通过嵌入层进行计算

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">
</p>

- 基于从raw text中提取的tokens可以创建一个由所有唯一的token组成的词语表/vocab

In [55]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


- 构建一个key为token，value为id数值的字典，即一个便于将token隐射到id的映射关系

In [56]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- 以下是词汇表中的前五十个实体

In [57]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 下图阐述了使用一个小的词汇表对一段短文进行分词的过程
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">
</p>


- 将上述关于分词的所有内容封装为一个类

In [58]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 替换指定标点符号前的空格
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- encode方法将文本转换为token IDs
- decode方法将token IDs转换会文本
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">
</p>

- 可使用分词器将文本编码为数值序列
- 数值序列进行嵌入化后作为LLM的输入

In [59]:
tokenizer = SimpleTokenizerV1(vocab)

text = """
           "It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride.
       """
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 将数值序列转换为文本

In [60]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [61]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊上下文tokens
- 在文本中可能会存在一些不知道的词汇，也需要一些特殊词汇表示文本结束等，就需要设置一些特殊tokens；下图中的|unk|、|endoftext|就是分别表示未知和文本结束的特殊tokens
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">
</p>

- 一些分词器使用特殊tokens帮助LLM理解额外的上下文，具体的特殊tokens有：
  - [BOS]：开始标记，表示文本的开始
  - [EOS]：结束标记，表示文本的结束(通常用于连接多个不相关的文本，例如两篇不同的维基百科文章或两本不同的书籍等）
  - [PAD]：LLMs训练的一个batch中，多条样本序列长度一般是不同的，一般会将所有样本长度都pad到最长的样本长度，就会使用[PAD]来填充
  - [UNK]：当文本中存在词汇表中不存在的token时，就用此未知token表示
- 注意：GPT-2不使用上述的任何特征字符，为了降低复杂度仅使用<|endoftext|>
  - <|endoftext|>相当于上述提到的[EOS]
  - GPT也是使用<|endoftext|>作为填充字符，因为训练过程计算注意力或损失时会使用mask排除padding的tokens，所以padding的token具体是什么其实不重要
- GPT-2不使用[UNK]表示词汇表外的单词；相反，GPT-2使用字节对编码（BPE）分词器，它将单词分解为子词单元，将在后面的章节中讨论这一点。
- 在两个独立、无关的文本之间使用<|endoftext|>进行区分
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">
</p>

- 看看分词以下文本会发生什么

In [62]:
tokenizer = SimpleTokenizerV1(vocab)
text = "Hello, do you like tea. Is this-- a test?"
tokenizer.encode(text)


KeyError: 'Hello'

- 看到产生了一个错误，因为词汇表中没有“Hello”这个token
- 为了处理这样的问题，可以将`"<UNK>"`这样的特殊token添加到词汇表中，当遇到词汇表中没有的token时，就用其表示
- 除了`"<UNK>"`外，还可以将<|endoftext|>一并添加，它在GPT-2训练中用来表示文本的结束（它也用于连接文本之间，比如当训练数据集包含多篇文章、书籍等时）。

In [63]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>", "<unk>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

len(vocab.items())

1132

In [64]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)


('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<unk>', 1131)


- 需要相应调整分词器/tokenizer，使其知道什么时候、应该如何使用<UNK>

In [65]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int else "<unk>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- 使用调整后的分词器对文本进行分词

In [66]:
tokenizer = SimpleTokenizerV2(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = text1 + " <|endoftext|> " + text2

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [67]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [68]:
tokenizer.decode(tokenizer.encode(text))

'<unk>, do you like tea? <|endoftext|> In the sunlit terraces of the <unk>.'

## 2.5 字节对编码(BytePair Encoding, BPE)
- GPT-2使用BPE作为分词器
- 它允许模型将不在其预定义词汇表中的单词分解为更小的子词单元甚至单个字符，使其能够处理词汇表外的单词
- 例如，如果GPT-2的词汇表没有"unfamiliarword"这个词，它可能会将其标记为["unfam", "iliar", "word"]或其他子词分解，这取决于它训练的BPE合并
- 原始的BPE分词器可以在这里找到：https://github.com/openai/gpt-2/blob/master/src/encoder.py
- 本章使用OpenAI开源的tiktoken库中的BPE分词器，它在Rust中实现其核心算法以提高计算性能
- 在[./bytepair_encoder](../02_bonus_bytepair-encoder)中创建了一个notebook，并排比较了这两种实现（在样本文本上，tiktoken大约快5倍）


In [69]:
# pip install tiktoken

In [70]:
import importlib
import tiktoken

print("tiktoken version: ", importlib.metadata.version("tiktoken"))

tiktoken version:  0.6.0


In [71]:
tokenizer = tiktoken.get_encoding("gpt2")

In [72]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [73]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE分词器将不知道的单词分解为子词或字符
<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="500px">
</p>


## 2.6 带滑动窗口的数据采样
- 训练大型语言模型(LLM)时是让它一次生成一个词，因此相应地准备训练数据，使序列中的下一个词代表预测的目标

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="600px">
</p>

In [74]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- 对于每个文本块，训练需要输入和对应的目标
- 因为需要模型去预测下一个词，目标是将输入向右移动一个位置得到的结果

In [75]:
enc_sample = enc_text[50:]

In [76]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 逐个预测的过程如下所示

In [77]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "----->", desired)

[290] -----> 4920
[290, 4920] -----> 2241
[290, 4920, 2241] -----> 287
[290, 4920, 2241, 287] -----> 257


- 将在介绍完注意力机制后的章节中处理下一个词的预测
- 现在，实现一个简单的数据加载器，它遍历输入数据集并返回输入以及向右移动一位的目标
- 首先要安装和导入pytorch

In [78]:
import torch
print("torch version: ", torch.__version__)

torch version:  2.2.1+cu118


- 使用滑动窗口方法，每次将位置向前移动+1

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">
</p>

- 创建数据集和数据加载器从文本数据中抽取块

In [79]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt: int, tokenizer: tiktoken.Encoding, max_length: int, stride: int):
        self.input_ids = []
        self.target_ids = []

        # 将文本进行完全分词
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        self.token_ids = token_ids
        
        # 使用滑动窗口将文本分割成最大长度为max_length的重叠序列
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
        
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [95]:
def create_dataloader_v1(txt: str, batch_size: int = 4, max_length: int = 256, stride: int = 128,
                         shuffle: bool = True, drop_last: bool = True, num_workers: int = 0) -> DataLoader:
    # 初始分词器
    tokenizer = tiktoken.get_encoding("gpt2")

    # 初始数据集
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 初始数据加载器
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers)

    return dataset, dataloader

- 以下使用批量大小为1，上下文大小为4来测试数据加载器

In [81]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [82]:
_, dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
print("数据集大小：", len(dataloader))

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
数据集大小： 5141


In [83]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 下面是使用步长等于上下文长度（这里是4）的示例

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">
</p>



- 也可以生成批量的结果
- 注意，这里增加了步长，这样批次之间就不会有重叠，因为更多的重叠可能导致过拟合增加

In [97]:
dataset, dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("文本分词后的前1~32个ids:\n", dataset.token_ids[:32])
print("Targets:\n", targets)
print("文本分词后的2~33个ids:\n", dataset.token_ids[1:33])

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
文本分词后的前1~32个ids:
 [40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438, 568, 340, 373, 645, 1049, 5975, 284, 502, 284, 3285, 326, 11]
Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
文本分词后的2~33个ids:
 [367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438, 568, 340, 373, 645, 1049, 5975, 284, 502, 284, 3285,

- 可以看到，当生成批次数据时，就是将多个max_length长度的序列子集组合在一个批次

## 2.7 生成token嵌入
- 通过以上处理，训练LLM的数据已准备好
- 接下来是使用嵌入层将token ids转换为连续可计算的向量表示
- 通常，嵌入层是LLM的一部分，在模型训练过程中会更新

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="500px">
</p>

- 假设有以下例子，文本转换后被分词为[2, 3, 5, 1]

In [85]:
input_ids = torch.tensor([2, 3, 5, 1])

- 简单起见，假设有一个只包含6个词的小词汇表，创建大小为3的嵌入

In [86]:
vocab_size = 6
embedding_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, embedding_dim)

- 上述会生成一个6×3的矩阵

In [87]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于熟悉独热编码(one-hot encoding)的人来说，上面的嵌入层方法本质上只是一种更高效的实现方式，相当于先进行独热编码然后在全连接层中进行矩阵乘法，这在[./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)的补充代码中有描述
- 因为嵌入层只是一种更高效的实现，等同于独热编码和矩阵乘法的方法，它可以被视为一个可以通过反向传播优化的神经网络层
- 将ID为3的标记转换为3维向量，可执行以下操作

In [88]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 注意，上述输出的是embedding_layer权重矩阵中的第四行
- 将整个input_ids整个转换为嵌入向量

In [89]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上是一个查找操作

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">
</p>

- **对比较嵌入层和常规线性层的额外内容感兴趣的读者参考：[../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 位置信息编码
- 无论IDs在输入序列中的位置如何，嵌入层会将相同的ID转换为相同的向量表示，需要一个额外的位置编码来表征各个token在序列中的位置信息

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="500px">
</p>

- 位置嵌入与标记嵌入向量相结合，形成LLM的输入嵌入

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">
</p>

- BytePair编码的词汇表大小为50,257
- 假设将输入的tokens转换为一个256维的向量表征

In [90]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 如果从dataloader中采样数据，将batch中的每个token都转换为256维的向量表征
- 如果batch是具有8个长度为4的tokens序列，即size为(8, 4)，则输出向量的size为(8, 4, 256)

In [99]:
max_length = 4
_, dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [100]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# 取消注释 & 执行以下行以查看嵌入向量是什么样的
# print(token_embeddings)

torch.Size([8, 4, 256])


- GPT-2使用绝对位置编码，生成另外一个嵌入层表示位置编码

In [101]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# 取消注释 & 执行以下行以查看位置嵌入层权重是什么样的
# print(pos_embedding_layer.weight)

In [102]:
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

# 取消注释 & 执行以下行以查看位置嵌入向量是什么样的
# print(pos_embeddings)

torch.Size([4, 256])


- 简单地将token嵌入和位置嵌入相加，就是LLM最终的输入

In [103]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# 取消注释 & 执行以下行以查看输入嵌入向量是什么样的
# print(input_embeddings)

torch.Size([8, 4, 256])


- 在输入处理流程的初始阶段，输入文本被划分为独立的tokens
- 划分之后，这些tokens基于预定义的词汇表转换为token IDs
- 最终，token IDs被转换为嵌入向量

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="500px">
</p>

# 总结和收获
- 请查看[./dataloader.ipynb](./dataloader.ipynb)代码笔记本，这是在本章实现的数据加载器的简洁版本，将在接下来的章节中用于训练GPT模型。
- 查看[./exercise-solutions.ipynb](./exercise-solutions.ipynb)获取练习解答。
- 如果有兴趣了解如何从头实现和训练GPT-2分词器，请查看[字节对编码(BPE)分词器从零开始](../05_bpe-from-scratch/bpe-from-scratch.ipynb)笔记本。
